## Data Preparation
---

### Movie Budgets

In [1]:
# Check the summary of the dataframe
movie_budgets.info()

In [ ]:
movie_budgets.head()

In [ ]:
# Merge the exploded imdb dataframe with the movie_budgets dataframe
imdb_revenue_exploded = imdb_exploded.merge(movie_budgets, left_on='primary_title', right_on='movie',how='inner')
imdb_revenue_exploded = imdb_revenue_exploded.drop(columns=['movie_id','movie'])
imdb_revenue_exploded.head()

In [ ]:
# Plot the distribution of movie budgets
sns.set_style('whitegrid')
sns.set_palette('bright')
plt.figure(figsize=(10, 5))
sns.histplot(movie_budgets['production_budget'], bins=30)
plt.axvline(movie_budgets['production_budget'].mean(), color='red', linestyle='dashed', linewidth=2, label='Mean')
plt.title('Distribution of Movie Budgets')
plt.xlabel('Budget Category')
plt.ylabel('Count')
plt.legend()
plt.show()

In [ ]:
#  Bin the production budget into three categories: Low, Medium, High
labels = ['Low', 'Medium', 'High']
bins = [0, 40000000, 100000000, movie_budgets['production_budget'].max()]
binned_budgets = movie_budgets.copy()
binned_budgets['budget_category'] = pd.cut(binned_budgets['production_budget'], bins=bins, labels=labels)
counts = binned_budgets.value_counts('budget_category').to_frame()
counts.reset_index(inplace=True)
counts

In [ ]:
# Create a visualization of the distribution of movie budgets
plt.figure(figsize=(10, 5))
sns.barplot(x='budget_category', y='count', data=counts, hue='budget_category', palette='Set1')
plt.title('Distribution of Movie Budgets')
plt.xlabel('Budget Category')
plt.ylabel('Count')
plt.show()

In [ ]:
# Create a visualization of the distribution of worldwide gross
plt.figure(figsize=(10, 5))
sns.histplot(movie_budgets['worldwide_gross'], bins=30)
plt.title('Distribution of Worldwide Gross')
plt.xlabel('Worldwide Gross')
plt.ylabel('Count')
plt.axvline(movie_budgets['worldwide_gross'].mean(), color='red', linestyle='dashed', linewidth=2, label='Mean')
plt.legend()
plt.show()

In [ ]:
# Bin the worldwide gross into three categories: Low, Medium, High
bins = [0, 25000000, 100000000, binned_budgets['worldwide_gross'].max()]
labels = ['Low', 'Medium', 'High']
binned_budgets['gross_category'] = pd.cut(binned_budgets['worldwide_gross'], bins=bins, labels=labels)
counts = binned_budgets.value_counts('gross_category').to_frame()
counts.reset_index(inplace=True)


In [ ]:
# Plot the distribution of worldwide gross categories
plt.figure(figsize=(10, 5))
sns.barplot(x='gross_category', y='count', data=counts, hue='gross_category', palette='Set1')
plt.title('Distribution of Worldwide Gross')
plt.xlabel('Gross Category')
plt.ylabel('Count')
plt.show()

### IMDB 

In [ ]:
# Assign the movie_basics info to movie_basics df
movie_basics = pd.read_sql("""
SELECT 
    movie_id, primary_title, genres
FROM movie_basics""",conn)
movie_basics.head()

In [ ]:
# Assign the ovie ratings info to movie_ratiings df
movie_ratings = pd.read_sql("""
SELECT
    movie_id, averagerating
FROM movie_ratings""",conn)
movie_ratings.head()

In [ ]:
# Assign the directors to directors df
directors = pd.read_sql("""
SELECT 
    movie_id, person_id, primary_name AS director_name
FROM directors
INNER JOIN persons
USING(person_id)
""",conn)
directors.head()

In [ ]:
# Assign actors in the database to a actors df
actors = pd.read_sql("""
SELECT 
    movie_id, person_id, primary_name AS actor_name
FROM principals
INNER JOIN persons
USING(person_id)
WHERE category = 'actor' 
""",conn)
actors.head()

In [ ]:
# Explode the genres column in the movie_basics DataFrame
movie_basics['genres'] = movie_basics['genres'].str.split(',')
movie_basics_exploded = movie_basics.explode('genres')

In [ ]:
# Merge the movie basics exploded df with the movie ratings df
imdb_exploded = movie_basics_exploded.merge(movie_ratings, on='movie_id', how='inner')
imdb_exploded.head()

In [ ]:
# Merge the exploded imdb dataframe with the movie_budgets dataframe
imdb_revenue_exploded = imdb_exploded.merge(movie_budgets, left_on='primary_title', right_on='movie',how='inner')

imdb_revenue_exploded.head()

In [ ]:
# Merge the imdb df with the directors df
imdb_directors = imdb_revenue_exploded.merge(directors, on='movie_id', how='inner')
imdb_directors = imdb_directors.drop(columns=['person_id'])
imdb_directors.head()

In [ ]:
# Merge the imdb_directors df with the actors df
imdb_crew = imdb_directors.merge(actors, on='movie_id', how='inner')
imdb_crew = imdb_crew.drop(columns=['person_id','movie'], axis=1)
imdb_crew.head()